In [1]:
import safetensors.torch as safe

l = safe.load_file

In [2]:
a = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/pretraindata-i-2d-801M-ctx8192/blocks.15.hook_resid_post/shard-0-chunk-00000000.safetensors"
)
a["tokens"].shape

torch.Size([1, 8192])

In [20]:
b = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/pretraindata-b-2d-1001M/blocks.15.hook_resid_post/shard-8-chunk-00000000.safetensors"
)
b["tokens"]

tensor([[128000,     48,     25,  ..., 128001, 128001, 128001],
        [128000,  25554,   4783,  ...,  51205,  68641,   3952],
        [128000,  13730,    315,  ...,    433,    574,   3062],
        [128000,  14422,    220,  ..., 128001, 128001, 128001]])

In [21]:
c = l(
    "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/reasondata-b-2d-801M-f/blocks.15.hook_resid_post/shard-15-chunk-00006111.safetensors"
)
c["tokens"].shape

torch.Size([1, 8192])

In [23]:
import os
from concurrent.futures import ProcessPoolExecutor

import safetensors.torch
from tqdm import tqdm

# 定义 A 路径
path_A = "/inspire/hdd/global_user/hezhengfu-240208120186/jiaxing_activations/pretraindata-b-2d-801M-ctx8192/blocks.15.hook_resid_post"  # 替换为实际的路径

# 获取所有 safetensors 文件的路径
files = []
for x in range(16):
    for y in range(6112):
        filename = f"shard-{x}-chunk-{y:08d}.safetensors"
        file_path = os.path.join(path_A, filename)
        if os.path.exists(file_path):
            files.append(file_path)


# 处理单个文件的函数
def process_file(file):
    # 加载 safetensors 文件
    data = safetensors.torch.load_file(file)
    tensor = data["tokens"]

    # 判断 tensor 的维度，如果是 2 维则在第0维加一个维度
    if len(tensor.shape) == 1:
        tensor = tensor.unsqueeze(0)

    # 将修改后的 tensor 保存到原位置
    # 更新字典中的 'tokens' 键
    data["tokens"] = tensor

    # 使用 safetensors 保存修改后的文件
    safetensors.torch.save_file(data, file)


# 使用 ProcessPoolExecutor 并行处理文件
with ProcessPoolExecutor(max_workers=64) as executor:
    list(tqdm(executor.map(process_file, files), total=len(files), desc="Processing safetensors files"))

Processing safetensors files:  18%|█▊        | 18090/97792 [03:54<17:13, 77.13it/s] 


KeyboardInterrupt: 